<a href="https://colab.research.google.com/github/chepalkalden/Artificial-Intellegence-Projects/blob/main/US%20Health-Care%20Claims%20Calssification/ML_MODEL_CLAIMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Load and Merge Datasets

In [ ]:
import pandas as pd

# Load cleaned datasets with specified dtypes to avoid DtypeWarning
beneficiary = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Claims-Categorization/data/cleaned_benificiary.csv", low_memory=False)
#carrier = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Claims-Categorization/data/cleaned_carrier.csv", dtype={'9': str, '10': str, '11': str, '46': str, '47': str, '48': str, '49': str, '50': str, '135': str, '136': str, '137': str, '138': str, '139': str, '140': str, '141': str}, low_memory=False)
inpatient = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Claims-Categorization/data/cleaned_inpatient.csv", low_memory=False)
outpatient = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Claims-Categorization/data/cleaned_outpatient.csv", dtype={'21': str, '23': str, '24': str, '25': str, '26': str, '27': str}, low_memory=False)
drug = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Claims-Categorization/data/cleaned_drug.csv", dtype={'3': str}, low_memory=False)


# Add claim_type column to each
#carrier['claim_type'] = 'carrier'
inpatient['claim_type'] = 'inpatient'
outpatient['claim_type'] = 'outpatient'
drug['claim_type'] = 'drug'

# Combine all claims
claims = pd.concat([inpatient, outpatient, drug], ignore_index=True)

# Merge with beneficiary data
#merged_df = claims.merge(beneficiary, on='DESYNPUF_ID', how='left')

In [ ]:
display(claims.head())

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,claim_type,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,00013D2EFD8E45D1,1.966612e+14,1.0,20100312.0,1970-01-01 00:00:00.020100313,1348.0,4000.0,0.0,3.139084e+09,NaN,...,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00016F745862898F,1.962012e+14,1.0,20090412.0,1970-01-01 00:00:00.020090418,1985.0,26000.0,0.0,6.476809e+09,NaN,...,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00016F745862898F,1.966612e+14,1.0,20090831.0,1970-01-01 00:00:00.020090902,1982.0,5000.0,0.0,6.119985e+08,6.119985e+08,...,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00016F745862898F,1.960912e+14,1.0,20090917.0,1970-01-01 00:00:00.020090920,2039.0,5000.0,0.0,4.971603e+09,NaN,...,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00016F745862898F,1.962612e+14,1.0,20100626.0,1970-01-01 00:00:00.020100701,1985.0,16000.0,0.0,6.408400e+09,1.960860e+09,...,inpatient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Step 2: Feature Selection & Engineering

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Select relevant features
features = ['DESYNPUF_ID', 'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2','ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5','ICD9_DGNS_CD_6','ICD9_DGNS_CD_7','ICD9_DGNS_CD_8','ICD9_DGNS_CD_9','ICD9_DGNS_CD_10','ICD9_PRCDR_CD_1','ICD9_PRCDR_CD_2','ICD9_PRCDR_CD_3','ICD9_PRCDR_CD_4','ICD9_PRCDR_CD_5','ICD9_PRCDR_CD_6', 'PRVDR_NUM', 'CLM_THRU_DT', 'CLM_PMT_AMT', 'claim_type']
df = claims[features].dropna()

# Encode date features
df['CLM_THRU_DT'] = pd.to_datetime(df['CLM_THRU_DT'], errors='coerce')
df['day_of_week'] = df['CLM_THRU_DT'].dt.dayofweek
df['month'] = df['CLM_THRU_DT'].dt.month

# Encode categorical features
for col in ['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2','ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5','ICD9_DGNS_CD_6','ICD9_DGNS_CD_7','ICD9_DGNS_CD_8','ICD9_DGNS_CD_9','ICD9_DGNS_CD_10','ICD9_PRCDR_CD_1','ICD9_PRCDR_CD_2','ICD9_PRCDR_CD_3','ICD9_PRCDR_CD_4','ICD9_PRCDR_CD_5','ICD9_PRCDR_CD_6',  'PRVDR_NUM']:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Final feature set
X = df[['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2','ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5','ICD9_DGNS_CD_6','ICD9_DGNS_CD_7','ICD9_DGNS_CD_8','ICD9_DGNS_CD_9','ICD9_DGNS_CD_10','ICD9_PRCDR_CD_1','ICD9_PRCDR_CD_2','ICD9_PRCDR_CD_3','ICD9_PRCDR_CD_4','ICD9_PRCDR_CD_5','ICD9_PRCDR_CD_6',  'PRVDR_NUM', 'CLM_PMT_AMT', 'day_of_week', 'month']]
y = LabelEncoder().fit_transform(df['claim_type'])


Feature Importance

In [ ]:
import matplotlib.pyplot as plt

importances = model.feature_importances_
feature_names = X.columns

plt.figure(figsize=(8, 5))
plt.barh(feature_names, importances)
plt.title("Feature Importance")
plt.xlabel("Importance Score")
plt.show()


Step 3: Train Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Cross-validation accuracy: {cv_scores.mean():.4f}")

# Evaluate
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation accuracy: 0.9914
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        87

    accuracy                           1.00        87
   macro avg       1.00      1.00      1.00        87
weighted avg       1.00      1.00      1.00        87

Confusion Matrix:
 [[87]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Linked inpatient, outpatient, and drug claims via DESYNPUF_ID

Selected high-impact features (diagnosis, procedure, provider, date, amount)

Trained and validated a Random Forest model

Evaluated performance with cross-validation and metrics